In [1]:
import numpy as np
import pandas as pd

## Setting up and preprocessing data

In [2]:
n = np.NaN  # setting non scores to np.Nan for better integration with numpy
rows_users_ = ['u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7']
columns_series = ['Vikings', 'Breaking Bad', 'The Sopranos', 'Westworld',
                  'Game of Thrones', 'The Witcher', 'Skam', 'Les bureau']
all_users = np.array([
    [5,5,n,3,4,n,n,n],
    [4,3,3,4,5,2,3,3],
    [1,3,3,2,2,3,4,1],
    [n,3,n,n,n,5,n,n],
    [3,4,n,4,5,5,1,n],
    [4,5,2,5,3,2,2,3],
    [1,1,n,n,2,1,n,n],
])
df = pd.DataFrame(all_users, columns=columns_series, index=rows_users_)
df

,Vikings,Breaking Bad,The Sopranos,Westworld,Game of Thrones,The Witcher,Skam,Les bureau
u1,5.0,5.0,NaN,3.0,4.0,NaN,NaN,NaN
u2,4.0,3.0,3.0,4.0,5.0,2.0,3.0,3.0
u3,1.0,3.0,3.0,2.0,2.0,3.0,4.0,1.0
u4,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN
u5,3.0,4.0,NaN,4.0,5.0,5.0,1.0,NaN
u6,4.0,5.0,2.0,5.0,3.0,2.0,2.0,3.0
u7,1.0,1.0,NaN,NaN,2.0,1.0,NaN,NaN


In [3]:
# calculating means for all users
users_mean = np.nanmean(all_users, axis=1)
df = pd.DataFrame(users_mean, index=rows_users_, columns=['means'])
df

,means
u1,4.250000
u2,3.375000
u3,2.375000
u4,4.000000
u5,3.666667
u6,3.250000
u7,1.250000


In [4]:
# calculating variance for all users
all_users_variance = np.empty(all_users.shape)
for i, user_scores in enumerate(all_users):
    all_users_variance[i] = user_scores - users_mean[i]
df = pd.DataFrame(all_users_variance, columns=columns_series, index=rows_users_)
df

,Vikings,Breaking Bad,The Sopranos,Westworld,Game of Thrones,The Witcher,Skam,Les bureau
u1,0.750000,0.750000,NaN,-1.250000,-0.250000,NaN,NaN,NaN
u2,0.625000,-0.375000,-0.375,0.625000,1.625000,-1.375000,-0.375000,-0.375
u3,-1.375000,0.625000,0.625,-0.375000,-0.375000,0.625000,1.625000,-1.375
u4,NaN,-1.000000,NaN,NaN,NaN,1.000000,NaN,NaN
u5,-0.666667,0.333333,NaN,0.333333,1.333333,1.333333,-2.666667,NaN
u6,0.750000,1.750000,-1.250,1.750000,-0.250000,-1.250000,-1.250000,-0.250
u7,-0.250000,-0.250000,NaN,NaN,0.750000,-0.250000,NaN,NaN


## Initializing active user

In [5]:
active_user = np.array([3, 2, 3, np.NaN, 4, 1, np.NaN, 5])
active_user_mean = np.nanmean(active_user)

In [6]:
# calculating variance for active user
active_users_variance = np.empty((1, 8))
for i, score in enumerate(active_user):
    active_users_variance[0, i] = score - active_user_mean
df = pd.DataFrame(active_users_variance)
df

,0,1,2,3,4,5,6,7
0,0.0,-1.0,0.0,NaN,1.0,-2.0,NaN,2.0


# Estimating new scores for active user
- Estimate the score this person is likely to give Westworld and Skam
- Solve the problem using Python and the corrcoef-function from Numpy

In [7]:
# calculating correlations between active user and all users
correlations = []
for other_users_variance in all_users_variance:
    temp_active = []
    temp_other = []
    for i in range(len(active_users_variance[0])):
        if not np.isnan(active_users_variance[0, i]) and not np.isnan(other_users_variance[i]):
            temp_active.append(active_users_variance[0, i])
            temp_other.append(other_users_variance[i])
    correlations_matrix = np.corrcoef(np.array(temp_active), np.array(temp_other))
    correlation = correlations_matrix[0][1]
    correlations.append(correlation)
df = pd.DataFrame(correlations)
df

,0
0,-8.660254e-01
1,5.477226e-01
2,-7.191950e-01
3,-1.000000e+00
4,-1.348400e-01
5,1.343055e-17
6,7.745967e-01


In [8]:
# function for estimating score of series for active user
def estimate_score(kappa, average_vote_a, product_index):
    sum_correlation_variance = 0
    j = product_index  # index of product we want to predict the vote for
    for i in range(len(all_users_variance)):
        if not np.isnan(all_users_variance[i][j]):
            print(all_users_variance[i][j],'\t\t\t\t', correlations[i])
            sum_correlation_variance += all_users_variance[i][j] * correlations[i]
    p_a_j = average_vote_a + kappa * sum_correlation_variance
    return round(p_a_j,2)

In [9]:
westworld = 3
est_west = estimate_score(1,active_user_mean,westworld)
print('Active users estimated score for westworld:', est_west)

-1.25 				 -0.8660254037844385
0.625 				 0.5477225575051661
-0.375 				 -0.7191949522280761
0.3333333333333335 				 -0.1348399724926484
1.75 				 1.3430553978047352e-17
Active users estimated score for westworld: 4.65


In [10]:
skam = 6
est_skam = estimate_score(1,active_user_mean,skam)
print('Active users estimated score for skam:', est_skam)

-0.375 				 0.5477225575051661
1.625 				 -0.7191949522280761
-2.6666666666666665 				 -0.1348399724926484
-1.25 				 1.3430553978047352e-17
Active users estimated score for skam: 1.99
